In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
file_path_train='/content/drive/My Drive/Colab Notebooks/task 2/CUB_200_2011/images_new/train'
file_path_test='/content/drive/My Drive/Colab Notebooks/task 2/CUB_200_2011/images_new/test'

In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms, datasets
from sklearn.metrics import classification_report,confusion_matrix
import numpy as np

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [7]:
trainset = torchvision.datasets.ImageFolder(root=file_path_train, transform=transform_train)
testset = torchvision.datasets.ImageFolder(root=file_path_test, transform=transform_test)

torch.manual_seed(42)

train_loader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=4)
test_loader  = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

classes = trainset.classes

# Print the number of images in each set
print('Number of images in training set:', len(trainset))
print('Number of images in test set:', len(testset))

Number of images in training set: 5994
Number of images in test set: 5794


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [8]:
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 200)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [9]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [11]:
# Train the model
num_epochs = 15
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    running_loss += loss.item()
    print('[Epoch %d] Loss: %.3f' %
          (epoch + 1, running_loss /100))

[Epoch 1] Loss: 9.726
[Epoch 2] Loss: 8.067
[Epoch 3] Loss: 6.689
[Epoch 4] Loss: 5.795
[Epoch 5] Loss: 5.067
[Epoch 6] Loss: 4.545
[Epoch 7] Loss: 4.111
[Epoch 8] Loss: 3.762
[Epoch 9] Loss: 3.541
[Epoch 10] Loss: 3.230
[Epoch 11] Loss: 3.111
[Epoch 12] Loss: 2.884
[Epoch 13] Loss: 2.740
[Epoch 14] Loss: 2.596
[Epoch 15] Loss: 2.499
[Epoch 16] Loss: 2.439
[Epoch 17] Loss: 2.364
[Epoch 18] Loss: 2.250
[Epoch 19] Loss: 2.143
[Epoch 20] Loss: 2.090


In [ ]:
torch.save(model.state_dict(), '/content/drive/My Drive/Colab Notebooks/task 2/CUB_200_2011/checkpoint/model.pt')

In [14]:
load_weight = torch.load('/content/drive/My Drive/Colab Notebooks/task 2/CUB_200_2011/checkpoint/model.pt')

In [15]:
# Load the state dictionary into the model
model.load_state_dict(load_weight)

<All keys matched successfully>

In [16]:
# Set the model to evaluation mode
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [17]:
y_corr=[]
y_pre=[]
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        y_corr.extend(labels)
        y_pre.extend(predicted)

In [18]:
ycorr=torch.tensor(y_corr).detach().cpu().numpy()
ypre=torch.tensor(y_pre).detach().cpu().numpy()

print(classification_report(ycorr, ypre))

              precision    recall  f1-score   support

           0       0.60      0.83      0.69        30
           1       0.77      0.77      0.77        30
           2       0.65      0.79      0.71        28
           3       0.79      0.87      0.83        30
           4       0.56      0.64      0.60        14
           5       0.71      0.91      0.80        11
           6       0.86      0.78      0.82        23
           7       0.85      0.61      0.71        18
           8       0.42      0.28      0.33        29
           9       0.96      0.87      0.91        30
          10       0.57      0.67      0.62        30
          11       1.00      1.00      1.00        26
          12       0.87      0.90      0.89        30
          13       0.89      0.83      0.86        30
          14       0.87      0.96      0.92        28
          15       1.00      0.93      0.96        28
          16       0.85      0.85      0.85        27
          17       1.00    

In [ ]:
confusion_matrix(ycorr, ypre)

array([[15, 11,  2, ...,  0,  0,  0],
       [ 1, 24,  1, ...,  0,  0,  0],
       [ 3,  2, 20, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ..., 21,  0,  0],
       [ 0,  0,  0, ...,  0, 22,  0],
       [ 0,  0,  0, ...,  0,  0, 25]])